In [1]:
# Disable Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import os

objc[45398]: Class CaptureDelegate is implemented in both /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11f7f6538) and /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x120940860). One of the two will be used. Which one is undefined.
objc[45398]: Class CVWindow is implemented in both /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11f7f6588) and /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11bafca68). One of the two will be used. Which one is undefined.
objc[45398]: Class CVView is implemented in both /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11f7f65b0) and /Volumes/SKK-T7/Apps/pip_envs/cvenv/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11bafca90). One of the two will be used. Which one is undefined.
objc[

In [3]:
# mediapose utils

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# VIDEO PATHS

In [4]:
vid1_path = os.path.join('data', 'sample_videos', 'curls3-1.mp4')
vid2_path = os.path.join('data', 'sample_videos', 'curls3-2.mp4')
vid3_path = os.path.join('data', 'sample_videos', 'curls3-3.mp4')

In [5]:
# right videos
r1_path = os.path.join('data', 'sample_videos', 'R1.mp4')
r2_path = os.path.join('data', 'sample_videos', 'R2.mp4')
r3_path = os.path.join('data', 'sample_videos', 'R3.mp4')
r4_path = os.path.join('data', 'sample_videos', 'R4.mp4')

# wrong videos
w1_path = os.path.join('data', 'sample_videos', 'W1.mp4')
w2_path = os.path.join('data', 'sample_videos', 'W2.mp4')
w3_path = os.path.join('data', 'sample_videos', 'W3.mp4')
w4_path = os.path.join('data', 'sample_videos', 'W4.mp4')

In [8]:
# right videos
rf_path = os.path.join('data', 'sample_videos', 'curls_R-F-1.mp4')
rl_path = os.path.join('data', 'sample_videos', 'curls_R-L-1.mp4')
rr_path = os.path.join('data', 'sample_videos', 'curls_R-R-1.mp4')

# wrong videos
wf_path = os.path.join('data', 'sample_videos', 'curls_W-F-1.mp4')
wl_path = os.path.join('data', 'sample_videos', 'curls_W-L-1.mp4')
wr_path = os.path.join('data', 'sample_videos', 'curls_W-R-1.mp4')

In [13]:
# frontraises
fr_f_path = os.path.join('data', 'sample_videos', 'fr_F.mp4')
fr_l_path = os.path.join('data', 'sample_videos', 'fr_L.mp4')
fr_r_path = os.path.join('data', 'sample_videos', 'fr_R.mp4')

# VERSION 7.1

In [9]:
# calculating the joint angles
def calculate_angle(a, b, c):
    # we will be calculating angle ABC
    
    # extract unit vector BA:
    v1 = a - b
    v1_u = v1/np.linalg.norm(v1)
    # extract unit vector BC:
    v2 = c - b
    v2_u = v2/np.linalg.norm(v2)

    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

## to create a helper function to extract (n, 18) dimensional array to store the coordinates of each landmark in each frame

## these are 16 tuples of keypoints to extract 18 relevant angles

keypoint_index = np.array([
    [33, 16, 14],
    [16, 14, 12],
    [14, 12, 11],
    [14, 12, 24],
    [ 0, 34, 12],
    [ 0, 34, 36],
    [12, 11, 13],
    [13, 11, 23],
    [11, 13, 15],
    [13, 15, 35],
    [12, 24, 26],
    [26, 24, 23],
    [24, 23, 25],
    [11, 23, 25],
    [24, 26, 28],
    [23, 25, 27],
    [26, 28, 32],
    [25, 27, 31]
])

def extract_angle_arr(input_video_path, keypoint_index = keypoint_index):
    angles = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass
            
            # we initialize to size (36,3) instead of (33,3) because we include 3 more key-points, for angle calculation
            frame_coordinates = np.zeros((37,3))
            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                frame_coordinates[i][0],frame_coordinates[i][1],frame_coordinates[i][2] = landmarks[i].x, landmarks[i].y, landmarks[i].z
            
            # left hand key-point
            frame_coordinates[33] = (frame_coordinates[18] + frame_coordinates[20])/2
            # right hand key-point
            frame_coordinates[35] = (frame_coordinates[17] + frame_coordinates[19])/2
            # neck key-point
            frame_coordinates[34] = (frame_coordinates[11] + frame_coordinates[12])/2
            # middle pelvis key-point
            frame_coordinates[36] = (frame_coordinates[23] + frame_coordinates[24])/2

            # initialize array to hold angles per frame
            angle = np.zeros(18)

            # extract angles based on the relevant key-points from the keypoint_index
            for i in range(18):
                angle[i] = calculate_angle(frame_coordinates[keypoint_index[i][0]], frame_coordinates[keypoint_index[i][1]], frame_coordinates[keypoint_index[i][2]])
            
            angles.append(angle)

    # Releasing the video capture device
    cap.release()

    return np.array(angles)






from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (n, 18) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr, senstivity = 1):
    
    
    # normalize the coordinates using sensitivity; higher sensitivity => more lenient scoring, lower sensitivity => strict scoring

    vid1_arr = vid1_arr/(np.linalg.norm(vid1_arr) * senstivity)
    vid2_arr = vid2_arr/(np.linalg.norm(vid2_arr) * senstivity)
    
    # calculate distance
    d = distance(vid1_arr, vid2_arr)
    # we give a score out of 100
    d_score = 100 - (d*100)
    
    return d_score






# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, senstivity=1):
    vid1_arr = extract_angle_arr(vid1_path)
    vid2_arr  = extract_angle_arr(vid2_path)

    score = compare_vid(vid1_arr, vid2_arr, senstivity)

    print("Score:", score)
    return score

### Comparing average score for Bicep Curls

In [10]:
r_path = [rf_path, rl_path, rr_path]
w_path = [wf_path, wl_path, wr_path]

In [15]:
r_score = 0
r_count = 0

for i in range(len(r_path)):
    for j in range(i+1, len(r_path)):
        r_score += combined_compare(r_path[i], r_path[j])
        r_count += 1

print("Average Right Video Comparison Score:", r_score/r_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 62.77854002852074
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 60.443947858380085
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 48.29008589082052
Average Right Video Comparison Score: 57.170857925907114


In [14]:
w_score = 0
w_count = 0

for i in range(len(r_path)):
    for j in range(len(w_path)):
        w_score += combined_compare(r_path[i], w_path[j])
        w_count += 1

print("Average Wrong Video Comparison Score:", w_score/w_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 81.52242039553799
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 65.37995407069654
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 60.588743967530334
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 58.808760608293106
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.46934935391677
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 48.49393124457495
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 52.04897088107992
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 48.64261177789526
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 73.7519158233781
Average Wrong Video Comparison Score: 63.52296201365588


### Comparing scores for Front Raise

In [16]:
combined_compare(fr_f_path, fr_l_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 67.91556020519107


67.91556020519107

In [17]:
combined_compare(fr_f_path, fr_r_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 68.26438485845838


68.26438485845838

In [18]:
combined_compare(fr_l_path, fr_r_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 71.37800428522779


71.37800428522779

# VERSION 7.2

In [19]:
# calculating the joint angles
def calculate_angle(a, b, c):
    # we will be calculating angle ABC
    
    # extract unit vector BA:
    v1 = a - b
    v1_u = v1/np.linalg.norm(v1)
    # extract unit vector BC:
    v2 = c - b
    v2_u = v2/np.linalg.norm(v2)

    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

## to create a helper function to extract (n, 18) dimensional array to store the coordinates of each landmark in each frame

## these are 18 tuples of keypoints to extract 18 relevant angles

keypoint_index = np.array([
    [33, 16, 14],
    [16, 14, 12],
    [14, 12, 11],
    [14, 12, 24],
    [ 0, 34, 12],
    [ 0, 34, 36],
    [12, 11, 13],
    [13, 11, 23],
    [11, 13, 15],
    [13, 15, 35],
    [12, 24, 26],
    [26, 24, 23],
    [24, 23, 25],
    [11, 23, 25],
    [24, 26, 28],
    [23, 25, 27],
    [26, 28, 32],
    [25, 27, 31]
])

def extract_angle_arr(input_video_path, keypoint_index = keypoint_index):
    angles = []
    
    cap = cv2.VideoCapture(input_video_path)

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while cap.isOpened():
            # Reading frames from live feed
            success, frame = cap.read()
            
            if not success:
                print("Ignoring empty camera frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # print(landmarks)
            except:
                pass
            
            # we initialize to size (36,3) instead of (33,3) because we include 3 more key-points, for angle calculation
            frame_coordinates = np.zeros((37,3))

            # inialize min-max coordinates variables
            x_min, y_min, z_min = np.inf, np.inf, np.inf
            x_max, y_max, z_max = -np.inf, -np.inf, -np.inf

            # extract (x,y,z) coordinate of each landmark
            for i in range(33):
                x, y, z = landmarks[i].x, landmarks[i].y, landmarks[i].z
                frame_coordinates[i][0], frame_coordinates[i][1], frame_coordinates[i][2] = x, y, z

                # checking and updating the min coordinates
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if z < z_min:
                    z_min = z
                
                # checking and updating the max coordinates
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
                if z > z_max:
                    z_max = z
            
            # left hand key-point
            frame_coordinates[33] = (frame_coordinates[18] + frame_coordinates[20])/2
            # right hand key-point
            frame_coordinates[35] = (frame_coordinates[17] + frame_coordinates[19])/2
            # neck key-point
            frame_coordinates[34] = (frame_coordinates[11] + frame_coordinates[12])/2
            # middle pelvis key-point
            frame_coordinates[36] = (frame_coordinates[23] + frame_coordinates[24])/2

            # normalize frame_coordinates
            for i in range(37):
                # normalizing x coordinate
                frame_coordinates[i][0] = (frame_coordinates[i][0] - x_min)/(x_max - x_min)
                # normalizing y coordinate
                frame_coordinates[i][1] = (frame_coordinates[i][1] - y_min)/(y_max - y_min)
                # normalizing z coordinate
                frame_coordinates[i][2] = (frame_coordinates[i][2] - z_min)/(z_max - z_min)

            # initialize array to hold angles per frame
            angle = np.zeros(18)

            # extract angles based on the relevant key-points from the keypoint_index
            for i in range(18):
                angle[i] = calculate_angle(frame_coordinates[keypoint_index[i][0]], frame_coordinates[keypoint_index[i][1]], frame_coordinates[keypoint_index[i][2]])
            
            angles.append(angle)

    # Releasing the video capture device
    cap.release()

    return np.array(angles)




from dtaidistance.dtw_ndim import distance, distance_fast

## to compare 2 videos of dim (n, 18) using DTW (using strategy 2)

def compare_vid(vid1_arr, vid2_arr, senstivity = 1):
    
    
    # normalize the coordinates using sensitivity; higher sensitivity => more lenient scoring, lower sensitivity => strict scoring

    vid1_arr = vid1_arr/(np.linalg.norm(vid1_arr) * senstivity)
    vid2_arr = vid2_arr/(np.linalg.norm(vid2_arr) * senstivity)
    
    # calculate distance
    d = distance(vid1_arr, vid2_arr)
    # we give a score out of 100
    d_score = 100 - (d*100)
    
    return d_score



# comparing videos by path: just including the pre-processing steps in the previous function

def combined_compare(vid1_path, vid2_path, senstivity=1):
    vid1_arr = extract_angle_arr(vid1_path)
    vid2_arr  = extract_angle_arr(vid2_path)

    score = compare_vid(vid1_arr, vid2_arr, senstivity)

    print("Score:", score)
    return score

In [20]:
r_score = 0
r_count = 0

for i in range(len(r_path)):
    for j in range(i+1, len(r_path)):
        r_score += combined_compare(r_path[i], r_path[j])
        r_count += 1

print("Average Right Video Comparison Score:", r_score/r_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 76.83928496785356
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 75.6500236414682
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 74.22925426625005
Average Right Video Comparison Score: 75.57285429185727


In [21]:
w_score = 0
w_count = 0

for i in range(len(r_path)):
    for j in range(len(w_path)):
        w_score += combined_compare(r_path[i], w_path[j])
        w_count += 1

print("Average Wrong Video Comparison Score:", w_score/w_count)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 82.9938427695738
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 74.89036478961378
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 70.63092097672896
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 72.04742862189255
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.57485928147821
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 70.45363820627182
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 66.03145508569847
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 70.3104276341097
Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 73.72010933335761
Average Wrong Video Comparison Score: 73.51700518874722


### Comparing scores for Front Raise

In [22]:
combined_compare(fr_f_path, fr_l_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 80.0525106255779


80.0525106255779

In [23]:
combined_compare(fr_f_path, fr_r_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 77.7673607806419


77.7673607806419

In [24]:
combined_compare(fr_l_path, fr_r_path)

Ignoring empty camera frame.
Ignoring empty camera frame.
Score: 83.18607247533338


83.18607247533338